In [1]:
from pyspark import SparkContext,SparkConf
from itertools import combinations
import time
import math

In [2]:
'''
input_file_path = sys.argv[1]
output_file_path = sys.argv[2]
'''
input_file_path = "/Users/gopi/Desktop/Assignment3/yelp_train.csv"
test_file_path = "/Users/gopi/Desktop/Assignment3/yelp_val_in.csv"
output_file_path = "/Users/gopi/Desktop/Assignment3/output_1.csv"
output_file_path_pairs = "/Users/gopi/Desktop/Assignment3/output_1_sim_m.csv"
#SparkContext.setSystemProperty('spark.executor.memory', '2g')
conf = SparkConf().setMaster("local[*]").setAppName("assign-3-task-1").set("spark.executor.memory", "4g").set("spark.driver.memory", "4g")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/20 18:28:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/20 18:28:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
def filter_pairs(user_ratings_1, user_ratings_2):
    if user_ratings_1 is None or user_ratings_2 is None:
        return False
    #print("set 1 is :", user_ratings_1)
    #print("set 2 is :", user_ratings_2)
    set_1 = set(user_ratings_1.keys())
    set_2 = set(user_ratings_2.keys())
    intersection = list(set_1.intersection(set_2))
    #print("inter is :", intersection)
    #print(len(intersection) > 0)
    return len(intersection) >= 1

def compute_similarity(user_ratings_1, user_ratings_2):
    set_1 = set(user_ratings_1.keys())
    set_2 = set(user_ratings_2.keys())
    corated = list(set_1.intersection(set_2))
    values_set_1 = list(user_ratings_1.values())
    values_set_2 = list(user_ratings_2.values())
    
    list_1 = []
    list_2 = []
    for user_id in corated:
        list_1.append(user_ratings_1[user_id])
        list_2.append(user_ratings_2[user_id])
    
    avg_1 = sum(list_1) / float(len(list_1))
    avg_2 = sum(list_2) / float(len(list_2))
    
    avg_1 = sum(values_set_1) / len(values_set_1)
    avg_2 = sum(values_set_2) / len(values_set_2)   
    
    num = 0
    denom_1 = 0
    denom_2 = 0
    for i in range(len(list_1)):
        num += ((list_1[i] - avg_1) * (list_2[i] - avg_2))
        denom_1 += ((list_1[i] - avg_1) ** 2)
        denom_2 += ((list_2[i] - avg_2) ** 2)
    
    denom = math.sqrt(denom_1) * math.sqrt(denom_2)
    if num == 0 or denom == 0:
        return 0
    
    return num / denom
    
    
def predict_rating(x):
    if x[0] not in user_index_map:
        return avg_business_rating[x[1]] if x[1] in business_index_map else 4.0
    
    if x[1] not in business_index_map:
        return avg_user_rating[x[0]] if x[0] in user_index_map else 4.0
    
    target_user_id = user_index_map[x[0]]
    target_business_id = business_index_map[x[1]]
    user_rated_business = user_business_rating_map[target_user_id]
    #limit = 5
    final = []
    for business in user_rated_business:
        business_id = business[0]
        key = (target_business_id, business_id) if target_business_id < business_id else (business_id, target_business_id)
        similarity = similarity_map[key] if key in similarity_map else 0.1
        #similarity = (similarity ** 2.5)
        if business_id != target_business_id:
            final.append((similarity, business[1]))
    
    #final = sorted(final, key = lambda x : x[0], reverse=True)
    #final = final[:limit]
    #print(final)
    num = 0
    denom = 0
    for pair in final:
        num += (pair[1] * pair[0])
        denom += pair[0]
    if denom == 0:
        return avg_user_rating[x[0]] if x[0] in user_index_map else 4.0
    return num / denom

def write_data_to_file(output_file_path, data):
    file = open(output_file_path, 'w')
    file.write("user_id, business_id, prediction")
    file.write("\n")
    for i in range(len(data)):
        key = data[i]
        rating = key[2]
        output = key[0] + "," + key[1] + "," + str(rating)
        file.write(output)
        file.write("\n")
    file.close()   
        

In [4]:
start = time.time()
initial_rdd = sc.textFile(input_file_path)
first = initial_rdd.first()
initial_rdd = initial_rdd.filter(lambda line : line != first).map(lambda line : line.split(",")).filter(lambda x : x[0] != None and x[1] != None and x[2] != None).map(lambda line: (line[0], line[1], float(line[2])))

user_index_map = initial_rdd.map(lambda x : x[0]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[0],x[1])).collectAsMap()
business_index_map = initial_rdd.map(lambda x : x[1]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[0],x[1])).collectAsMap()

index_user_map = initial_rdd.map(lambda x : x[0]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[1],x[0])).collectAsMap() 
index_business_map = initial_rdd.map(lambda x : x[1]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[1],x[0])).collectAsMap()

test_data_rdd = sc.textFile(test_file_path)
first = test_data_rdd.first()
test_data_rdd = test_data_rdd.filter(lambda x : x != first).map(lambda line : line.split(","))
business_id_set = set(test_data_rdd.map(lambda x : x[1]).collect())

filtered_input = initial_rdd.map(lambda x: (business_index_map[x[1]], (user_index_map[x[0]], x[2]))).groupByKey().mapValues(list).mapValues(lambda x : dict(x))
business_user_rating_map = filtered_input.collectAsMap()

user_business_rating_map = initial_rdd.map(lambda x: (user_index_map[x[0]], (business_index_map[x[1]], x[2]))).groupByKey().mapValues(list).collectAsMap()

business_items = filtered_input.map(lambda x: x[0])

business_pairs = business_items.cartesian(business_items).filter(lambda x : x[0] < x[1]).filter(lambda x : x[0] in business_id_set).filter(lambda x : filter_pairs(business_user_rating_map[x[0]], business_user_rating_map[x[1]])).partitionBy(15)
similarity_map = business_pairs.map(lambda x : (x, compute_similarity(business_user_rating_map[x[0]], business_user_rating_map[x[1]]))).filter(lambda x : x[1] > 0.01).collectAsMap()




/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with s

In [5]:
avg_business_rating = initial_rdd.map(lambda x : (x[1], float(x[2]))).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1])/len(x[1]))).collectAsMap()
avg_user_rating = initial_rdd.map(lambda x : (x[0], float(x[2]))).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1])/len(x[1]))).collectAsMap()

results_rdd = test_data_rdd.map(lambda x : (x[0], x[1], predict_rating(x))).collect()
write_data_to_file(output_file_path, results_rdd)

/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [6]:
import csv
def compute_rmse():

    file_2 = open("/Users/gopi/Desktop/Assignment3/yelp_val.csv", "r")
    csv_2 = csv.reader(file_2, delimiter=",")
    
    pred_1 = []
    pred_2 = []
    c = 0
    with open("/Users/gopi/Desktop/Assignment3/output_1.csv") as file:
        csv_1 = csv.reader(file, delimiter=",")
        for row in csv_1:
            if c != 0:
                pred_1.append(float(row[2]))
                
            c+=1
        c=0        
    for row in csv_2:
        if c != 0:
            pred_2.append(float(row[2]))
        c+=1
            
    total = 0
    for i in range(len(pred_1)):
        total += ((pred_1[i] - pred_2[i]) ** 2)
    
    return math.sqrt(total/len(pred_1))

print(compute_rmse())

1.0754449514566038


In [ ]:
import csv
def get_hybrid_prediction(pred_1, pred_2):
    for i in range(len(pred_1)):
        final_pred[i].append((0.9 * pred_2[i]) + (0.1 * pred_1[i]))
write_data_to_file("/Users/gopi/Desktop/Assignment3/output_2_3.csv", final_pred)

In [34]:
def get_neighbours_count(x):
    if x[0] not in user_index_map:
        return (x[0], 0)
    
    if x[1] not in business_index_map:
        return (x[0],0)
    
    target_user_id = user_index_map[x[0]]
    target_business_id = business_index_map[x[1]]
    user_rated_business = user_business_rating_map[target_user_id]
    #limit = 5
    final = []
    for business in user_rated_business:
        business_id = business[0]
        key = (target_business_id, business_id) if target_business_id < business_id else (business_id, target_business_id)
        similarity = similarity_map[key] if key in similarity_map else 0.1
        #similarity = (similarity ** 2.5)
        if business_id != target_business_id:
            final.append((similarity, business[1]))     
    return  (x[0], len(final))

n_count_rdd = test_data_rdd.map(lambda x : get_neighbours_count(x)).collect()


In [38]:
count = 0
n_count_rdd = sorted(n_count_rdd, key = lambda x : x[1], reverse=False)
a = []
for value in n_count_rdd:
    a.append(value[1])
print(sorted(list(set(a))))

[0, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 171, 172, 174, 175, 177, 178, 179, 180, 181, 182, 183, 185, 186, 187, 189, 190, 191, 192, 193, 196, 197, 198, 199, 200, 201, 202, 203, 205, 206, 208, 209, 210, 214, 216, 217, 220, 221, 222, 223, 226, 227, 230, 231, 233, 234, 235, 237, 238, 239, 240, 246, 249, 250, 252, 254, 256, 258, 261, 262, 264, 265, 267, 269, 271, 273